# Parte 3
## Radom Forest

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
# Hypter-parametros
from sklearn.model_selection import RandomizedSearchCV
# Metrica 
from sklearn.metrics import roc_auc_score
# Encoder 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
# CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# Seteo una seed para que el modelo sea reproducible
np.random.seed(seed=27)

In [3]:
train_df = pd.read_parquet('/content/drive/MyDrive/Orga Datos/TP3/TP3-OK/train.parquet')
test_df = pd.read_parquet('/content/drive/MyDrive/Orga Datos/TP3/TP3-OK/test.parquet')

In [4]:
train_df.content.fillna(' ', inplace=True)
test_df.content.fillna(' ', inplace=True)

In [5]:
X = train_df[train_df.columns[2:-2]] # no incluyo la url ni las cols popular ni shares
y = train_df[train_df.columns[-2]] # col popular (la que quiero predecir)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Utilizo CountVectorizer para encodear

In [8]:
stop_words = set(stopwords.words('english'))

In [9]:
content_list_X_train = X_train['content'].to_list()
count_vectorizer = TfidfVectorizer(lowercase=True, stop_words=stop_words, analyzer="word", max_features=5)

In [10]:
tf_matrix_X_train = count_vectorizer.fit_transform(content_list_X_train)
tf_array_X_train = tf_matrix_X_train.toarray()

In [11]:
most_common_words_X_train_df = pd.DataFrame(tf_array_X_train, columns=count_vectorizer.get_feature_names_out())

In [12]:
X_train = pd.concat([X_train.reset_index(), most_common_words_X_train_df.reset_index()], axis=1)

In [13]:
X_train = X_train.drop(columns=['index', 'content'])

In [14]:
content_list_X_test = X_test['content'].to_list()

In [15]:
tf_matrix_X_test = count_vectorizer.transform(content_list_X_test)
tf_array_X_test = tf_matrix_X_test.toarray()

In [16]:
most_common_words_X_test_df = pd.DataFrame(tf_array_X_test, columns=count_vectorizer.get_feature_names_out())

In [17]:
X_test = pd.concat([X_test.reset_index(), most_common_words_X_test_df.reset_index()], axis=1)

In [18]:
X_test = X_test.drop(columns=['index', 'content'])

In [19]:
# Encodeo surprise1

In [20]:
oho_surprise1_encoder = OneHotEncoder(handle_unknown='ignore')
encoded_surprise1_train = oho_surprise1_encoder.fit_transform(X_train[['surprise1']]).todense().astype(int)
encoded_surprise1_test = oho_surprise1_encoder.transform(X_test[['surprise1']]).todense().astype(int)

In [21]:
oho_categories_surprise1_X_train_df = pd.DataFrame(encoded_surprise1_train, columns=oho_surprise1_encoder.categories_)

In [22]:
X_train = pd.concat([X_train.reset_index(), oho_categories_surprise1_X_train_df], axis=1)

In [23]:
X_train = X_train.drop(columns=['index', 'surprise1'])

In [24]:
oho_categories_surprise1_X_test_df = pd.DataFrame(encoded_surprise1_test, columns=oho_surprise1_encoder.categories_)

In [25]:
X_test = pd.concat([X_test.reset_index(), oho_categories_surprise1_X_test_df], axis=1)

In [26]:
X_test = X_test.drop(columns=['index', 'surprise1'])

In [27]:
# Encodeo surprise2

In [28]:
oho_surprise2_encoder = OneHotEncoder(handle_unknown='ignore')
encoded_surprise2_train = oho_surprise2_encoder.fit_transform(X_train[['surprise2']]).todense().astype(int)
encoded_surprise2_test = oho_surprise2_encoder.transform(X_test[['surprise2']]).todense().astype(int)

In [29]:
oho_categories_surprise2_X_train_df = pd.DataFrame(encoded_surprise2_train, columns=oho_surprise2_encoder.categories_)

In [30]:
X_train = pd.concat([X_train.reset_index(), oho_categories_surprise2_X_train_df], axis=1)

In [31]:
X_train = X_train.drop(columns=['index', 'surprise2'])

In [32]:
oho_categories_surprise2_X_test_df = pd.DataFrame(encoded_surprise2_test, columns=oho_surprise2_encoder.categories_)

In [33]:
X_test = pd.concat([X_test.reset_index(), oho_categories_surprise2_X_test_df], axis=1)

In [34]:
X_test = X_test.drop(columns=['index', 'surprise2'])

In [35]:
# Remplazo los Nan

In [36]:
def llenar_nulos(df):
  df['n_tokens_title'].fillna(df['n_tokens_title'].mean(), inplace=True)
  df['n_tokens_content'].fillna(df['n_tokens_content'].mean(), inplace=True)
  df['n_non_stop_unique_tokens'].fillna(df['n_non_stop_unique_tokens'].mean(), inplace=True)
  df['num_hrefs'].fillna(float(df['num_hrefs'].mode()), inplace=True)
  df['num_self_hrefs'].fillna(float(df['num_self_hrefs'].mode()), inplace=True)
  df['num_imgs'].fillna(df['num_imgs'].min(), inplace=True)
  df['num_videos'].fillna(df['num_videos'].min(), inplace=True)
  df['LDA_00'].fillna(df['LDA_00'].mean(), inplace=True)
  df['LDA_01'].fillna(df['LDA_01'].mean(), inplace=True)
  df['LDA_02'].fillna(df['LDA_02'].mean(), inplace=True)
  df['LDA_03'].fillna(df['LDA_03'].mean(), inplace=True)
  df['kw_min_min'].fillna(float(df['kw_min_min'].mode()), inplace=True)
  df['kw_max_min'].fillna(float(df['kw_max_min'].mode()), inplace=True)
  df['kw_min_avg'].fillna(float(df['kw_min_avg'].mode()), inplace=True)
  df['kw_avg_min'].fillna(float(df['kw_avg_min'].mode()), inplace=True)
  df['kw_min_max'].fillna(float(df['kw_min_max'].mode()), inplace=True)
  df['kw_max_max'].fillna(float(df['kw_max_max'].mode()), inplace=True)
  df['kw_min_avg'].fillna(float(df['kw_min_avg'].mode()), inplace=True)
  df['self_reference_avg_sharess'].fillna(float(df['self_reference_avg_sharess'].mode()), inplace=True)
  df['max_positive_polarity'].fillna(df['max_positive_polarity'].max(), inplace=True)

  df.fillna(0.0, inplace=True)

In [37]:
llenar_nulos(X_train)
llenar_nulos(X_test)

In [38]:
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.2, random_state=42)

In [39]:
modelRandomFor = RandomForestClassifier()

In [ ]:
param_grid = [
  { 'n_estimators': [10, 50, 100],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['log2', 'sqrt'],
  }
]
clf = RandomizedSearchCV(modelRandomFor, param_grid, cv = 2, n_jobs=1, n_iter=3)
best_clf = clf.fit(X_train, y_train)

In [ ]:
predictions = best_clf.predict_proba(X_valid)[:,1]

In [42]:
metric = roc_auc_score(y_valid, predictions)
print("La metrica del modelo para validacion es: ", metric)

La metrica del modelo para validacion es:  0.7073584627473424


## Entre Random Forest y XGBoost me quedo con Radom como mejor modelo

In [43]:
x_test = test_df[test_df.columns[2:-2]]
y_test = test_df[test_df.columns[-2]]

In [44]:
content_list_x_test = x_test['content'].to_list()

In [45]:
tf_matrix_x_test = count_vectorizer.transform(content_list_x_test)
tf_array_x_test = tf_matrix_x_test.toarray()

In [46]:
most_common_words_x_test_df = pd.DataFrame(tf_array_x_test, columns=count_vectorizer.get_feature_names_out())

In [47]:
x_test = pd.concat([x_test.reset_index(), most_common_words_x_test_df.reset_index()], axis=1)

In [48]:
x_test = x_test.drop(columns=['index', 'content'])

In [49]:
# Surprise 1 y 2

In [50]:
encoded_surprise1_x_test = oho_surprise1_encoder.transform(x_test[['surprise1']]).todense().astype(int)
encoded_surprise2_x_test = oho_surprise2_encoder.transform(x_test[['surprise2']]).todense().astype(int)

In [51]:
oho_categories_surprise1_x_test_df = pd.DataFrame(encoded_surprise1_x_test, columns=oho_surprise1_encoder.categories_)
oho_categories_surprise2_x_test_df = pd.DataFrame(encoded_surprise2_x_test, columns=oho_surprise2_encoder.categories_)

In [52]:
x_test = pd.concat([x_test.reset_index(), oho_categories_surprise1_x_test_df], axis=1)
x_test = pd.concat([x_test.reset_index(), oho_categories_surprise2_x_test_df], axis=1)

In [53]:
x_test = x_test.drop(columns=['level_0', 'index', 'surprise1', 'surprise2'])

In [54]:
x_test['LDA_00'].fillna(x_test['LDA_00'].mean(), inplace=True)
x_test['LDA_01'].fillna(x_test['LDA_01'].mean(), inplace=True)
x_test['LDA_02'].fillna(x_test['LDA_02'].mean(), inplace=True)
x_test['LDA_03'].fillna(x_test['LDA_03'].mean(), inplace=True)
x_test.fillna(0.0, inplace=True)

In [ ]:
predictions = best_clf.predict_proba(x_test)[:,1]

In [56]:
metric = roc_auc_score(y_test, predictions)
print("La metrica del modelo para validacion es: ", metric)

La metrica del modelo para validacion es:  0.7260299020074477


In [57]:
scores_test = pd.DataFrame(best_clf.cv_results_)
scores_test.to_csv('/content/drive/MyDrive/Orga Datos/TP3/TP3-OK/scores_RandomFor_test.csv')